In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Activation
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, mean_squared_error
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from tensorflow.keras.optimizers import Adam

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
data_path = '/content/drive/MyDrive/Statistical2/Lab2/supermarket_sales.csv'
data = pd.read_csv(data_path)
data.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


In [ ]:
data.shape

(1000, 17)

## Feature Engineering

In [ ]:
data = data.drop(['Invoice ID', 'Date', 'Time', 'Tax 5%', 'Total', 'cogs', 'gross margin percentage', 'gross income'], axis = 1)
data.head()

,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Payment,Rating
0,A,Yangon,Member,Female,Health and beauty,74.69,7,Ewallet,9.1
1,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,Cash,9.6
2,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,Credit card,7.4
3,A,Yangon,Member,Male,Health and beauty,58.22,8,Ewallet,8.4
4,A,Yangon,Normal,Male,Sports and travel,86.31,7,Ewallet,5.3


In [ ]:
data.isnull().sum()

Branch           0
City             0
Customer type    0
Gender           0
Product line     0
Unit price       0
Quantity         0
Payment          0
Rating           0
dtype: int64

In [ ]:
def categorical_cols(df):
    colnames = df.columns
    categorical_vars = []
    for col in colnames:
        if(df[col].dtype == 'object'):
            categorical_vars.append(col)
    return categorical_vars

categorical_cols(data)

['Branch', 'City', 'Customer type', 'Gender', 'Product line', 'Payment']

In [ ]:
def labelencodecolumns(df):
  colnames = df.columns
  for col in colnames:
    if(df[col].dtype == 'object'):
      data[col] = data[col].astype('category')
      data[col] = data[col].cat.codes
  return (df.head())

labelencodecolumns(data)

,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Payment,Rating
0,0,2,0,0,3,74.69,7,2,9.1
1,2,1,1,0,0,15.28,5,0,9.6
2,0,2,1,1,4,46.33,7,1,7.4
3,0,2,0,1,3,58.22,8,2,8.4
4,0,2,1,1,5,86.31,7,2,5.3



## Data para entrenamiento y prueba


In [ ]:
X = data.iloc[:, data.columns != 'Quantity']
y = data.loc[:, 'Quantity']

#partición de data para entrenamiento y prueba.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state= 123)
std_scaler = StandardScaler()
X_train = std_scaler.fit_transform(X_train)
X_test = std_scaler.fit_transform(X_test)

X_train.shape, X_test.shape

((800, 8), (200, 8))

In [ ]:
## Parametros
parametros = {
    'batch_size': [50, 100],
    'optimizador': ['adam', 'rmsprop'],
    'epochs': [10, 25, 50],
}

### Estructura 1

In [ ]:
# Estructura 1
def modelo(optimizador):
  regressor = Sequential()
  regressor.add(Dense(input_dim= 8, units=7, kernel_initializer='uniform', activation='relu'))
  regressor.add(Dense(units=14, kernel_initializer='uniform', activation='relu'))
  regressor.add(Dense(units=7, kernel_initializer='uniform', activation='relu'))
  regressor.add(Dense(units=1, kernel_initializer='uniform', activation='relu'))
  regressor.compile(optimizer=optimizador, loss='mean_squared_error', metrics=['mean_squared_error'])
  return regressor

In [ ]:
resultado = KerasRegressor(modelo, validation_split=0.2, verbose=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  """Entry point for launching an IPython kernel.


In [ ]:
grid_search = GridSearchCV(estimator= resultado, 
                         param_grid= parametros, 
                         scoring='neg_root_mean_squared_error', 
                         cv=5)

In [ ]:
grid_search.fit(X_train, y_train, verbose = 1)

Epoch 1/10
11/11 [==============================] - 1s 49ms/step - loss: 38.4061 - mean_squared_error: 38.4061 - val_loss: 38.6518 - val_mean_squared_error: 38.6518
Epoch 2/10
11/11 [==============================] - 0s 8ms/step - loss: 38.2909 - mean_squared_error: 38.2909 - val_loss: 38.5087 - val_mean_squared_error: 38.5087
Epoch 3/10
11/11 [==============================] - 0s 12ms/step - loss: 38.1348 - mean_squared_error: 38.1348 - val_loss: 38.3387 - val_mean_squared_error: 38.3387
Epoch 4/10
11/11 [==============================] - 0s 10ms/step - loss: 37.9500 - mean_squared_error: 37.9500 - val_loss: 38.1279 - val_mean_squared_error: 38.1279
Epoch 5/10
11/11 [==============================] - 0s 8ms/step - loss: 37.7114 - mean_squared_error: 37.7114 - val_loss: 37.8525 - val_mean_squared_error: 37.8525
Epoch 6/10
11/11 [==============================] - 0s 9ms/step - loss: 37.3916 - mean_squared_error: 37.3916 - val_loss: 37.4662 - val_mean_squared_error: 37.4662
Epoch 7/10
11

GridSearchCV(cv=5,
             estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x7fcf4d22ea10>,
             param_grid={'batch_size': [50, 100], 'epochs': [10, 25, 50],
                         'optimizador': ['adam', 'rmsprop']},
             scoring='neg_root_mean_squared_error')

In [ ]:
grid_search.best_params_

{'batch_size': 50, 'epochs': 50, 'optimizador': 'rmsprop'}

In [ ]:
regressor = Sequential()
regressor.add(Dense(input_dim= 8, units=7, kernel_initializer='uniform', activation='relu'))
regressor.add(Dense(units=14, kernel_initializer='uniform', activation='relu'))
regressor.add(Dense(units=7, kernel_initializer='uniform', activation='relu'))
regressor.add(Dense(units=1, kernel_initializer='uniform', activation='relu'))
regressor.compile(optimizer='rmsprop', loss='mean_squared_error', metrics=['mean_squared_error'])
regressor.fit(X_train, y_train, batch_size=50, validation_split=0.2, verbose=1, epochs=50)

Epoch 1/50
13/13 [==============================] - 1s 15ms/step - loss: 38.9828 - mean_squared_error: 38.9828 - val_loss: 37.6036 - val_mean_squared_error: 37.6036
Epoch 2/50
13/13 [==============================] - 0s 4ms/step - loss: 38.7368 - mean_squared_error: 38.7368 - val_loss: 37.3731 - val_mean_squared_error: 37.3731
Epoch 3/50
13/13 [==============================] - 0s 4ms/step - loss: 38.4778 - mean_squared_error: 38.4777 - val_loss: 37.0970 - val_mean_squared_error: 37.0970
Epoch 4/50
13/13 [==============================] - 0s 5ms/step - loss: 38.1586 - mean_squared_error: 38.1586 - val_loss: 36.7547 - val_mean_squared_error: 36.7547
Epoch 5/50
13/13 [==============================] - 0s 4ms/step - loss: 37.7626 - mean_squared_error: 37.7626 - val_loss: 36.3300 - val_mean_squared_error: 36.3300
Epoch 6/50
13/13 [==============================] - 0s 5ms/step - loss: 37.2622 - mean_squared_error: 37.2622 - val_loss: 35.7998 - val_mean_squared_error: 35.7998
Epoch 7/50
13/1

In [ ]:
path = '/content/drive/MyDrive/Statistical2/Lab2/' 
regressor.save(path + '7_14_7_rmsprop.h5')

In [ ]:
model_1 = keras.models.load_model(path + '7_14_7_rmsprop.h5')

In [ ]:
y_preds = model_1.predict(X_test)
print(mean_squared_error(y_test, y_preds))

9.406463403763839


### Estructura 2

In [ ]:
# Estructura 2
def modelo(optimizador):
  regressor2 = Sequential()
  regressor2.add(Dense(input_dim= 8, units=4, kernel_initializer='normal', activation='relu'))
  regressor2.add(Dense(units=8, kernel_initializer='normal', activation='relu'))
  regressor2.add(Dense(units=4, kernel_initializer='normal', activation='relu'))
  regressor2.add(Dense(units=1, kernel_initializer='normal', activation='relu'))
  regressor2.compile(optimizer=optimizador, loss='mean_squared_error', metrics=['mean_squared_error'])
  return regressor2

In [ ]:
resultado2 = KerasRegressor(modelo, validation_split=0.2, verbose=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  """Entry point for launching an IPython kernel.


In [ ]:
grid_search2 = GridSearchCV(estimator= resultado2, 
                         param_grid= parametros, 
                         scoring='neg_root_mean_squared_error', 
                         cv=5)

In [ ]:
grid_search2.fit(X_train, y_train, verbose = 1)

Epoch 1/10
11/11 [==============================] - 1s 16ms/step - loss: 38.3750 - mean_squared_error: 38.3750 - val_loss: 38.6060 - val_mean_squared_error: 38.6060
Epoch 2/10
11/11 [==============================] - 0s 7ms/step - loss: 38.2490 - mean_squared_error: 38.2490 - val_loss: 38.4740 - val_mean_squared_error: 38.4740
Epoch 3/10
11/11 [==============================] - 0s 4ms/step - loss: 38.1119 - mean_squared_error: 38.1119 - val_loss: 38.3340 - val_mean_squared_error: 38.3340
Epoch 4/10
11/11 [==============================] - 0s 4ms/step - loss: 37.9686 - mean_squared_error: 37.9686 - val_loss: 38.1798 - val_mean_squared_error: 38.1798
Epoch 5/10
11/11 [==============================] - 0s 6ms/step - loss: 37.8071 - mean_squared_error: 37.8071 - val_loss: 38.0016 - val_mean_squared_error: 38.0016
Epoch 6/10
11/11 [==============================] - 0s 6ms/step - loss: 37.6123 - mean_squared_error: 37.6123 - val_loss: 37.7783 - val_mean_squared_error: 37.7783
Epoch 7/10
11/1

GridSearchCV(cv=5,
             estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x7fcf37d60690>,
             param_grid={'batch_size': [50, 100], 'epochs': [10, 25, 50],
                         'optimizador': ['adam', 'rmsprop']},
             scoring='neg_root_mean_squared_error')

In [ ]:
grid_search2.best_params_

{'batch_size': 50, 'epochs': 50, 'optimizador': 'adam'}

In [ ]:
regressor2 = Sequential()
regressor2.add(Dense(input_dim= 8, units=4, kernel_initializer='normal', activation='relu'))
regressor2.add(Dense(units=8, kernel_initializer='normal', activation='relu'))
regressor2.add(Dense(units=4, kernel_initializer='normal', activation='relu'))
regressor2.add(Dense(units=1, kernel_initializer='normal', activation='relu'))
regressor2.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])
regressor2.fit(X_train, y_train, batch_size=50, validation_split=0.2, verbose=1, epochs=50)

Epoch 1/50
13/13 [==============================] - 1s 16ms/step - loss: 39.1360 - mean_squared_error: 39.1360 - val_loss: 37.8584 - val_mean_squared_error: 37.8584
Epoch 2/50
13/13 [==============================] - 0s 4ms/step - loss: 39.0370 - mean_squared_error: 39.0370 - val_loss: 37.7071 - val_mean_squared_error: 37.7071
Epoch 3/50
13/13 [==============================] - 0s 4ms/step - loss: 38.8673 - mean_squared_error: 38.8673 - val_loss: 37.5284 - val_mean_squared_error: 37.5284
Epoch 4/50
13/13 [==============================] - 0s 4ms/step - loss: 38.6716 - mean_squared_error: 38.6716 - val_loss: 37.3258 - val_mean_squared_error: 37.3258
Epoch 5/50
13/13 [==============================] - 0s 4ms/step - loss: 38.4451 - mean_squared_error: 38.4451 - val_loss: 37.0830 - val_mean_squared_error: 37.0830
Epoch 6/50
13/13 [==============================] - 0s 7ms/step - loss: 38.1679 - mean_squared_error: 38.1679 - val_loss: 36.7756 - val_mean_squared_error: 36.7756
Epoch 7/50
13/1

In [ ]:
path = '/content/drive/MyDrive/Statistical2/Lab2/' 
regressor2.save(path + '4_8_4_adam.h5')

In [ ]:
model_2 = keras.models.load_model(path + '4_8_4_adam.h5')

In [ ]:
y_preds2 = model_2.predict(X_test)
print(mean_squared_error(y_test, y_preds2))

9.25782349632332


### Estructura 3

In [ ]:
# Estructura 3
def modelo(optimizador):
  regressor3 = Sequential()
  regressor3.add(Dense(input_dim= 8, units=8, kernel_initializer='normal', activation='selu'))
  regressor3.add(Dense(units=10, kernel_initializer='normal', activation='selu'))
  regressor3.add(Dense(units=4, kernel_initializer='normal', activation='selu'))
  regressor3.add(Dense(units=1, kernel_initializer='normal', activation='relu'))
  regressor3.compile(optimizer=optimizador, loss='mean_squared_error', metrics=['mean_squared_error'])
  return regressor3

In [ ]:
resultado3 = KerasRegressor(modelo, validation_split=0.2, verbose=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  """Entry point for launching an IPython kernel.


In [ ]:
grid_search3 = GridSearchCV(estimator= resultado3, 
                         param_grid= parametros, 
                         scoring='neg_root_mean_squared_error', 
                         cv=5)

In [ ]:
grid_search3.fit(X_train, y_train, verbose = 1)

Epoch 1/10
11/11 [==============================] - 1s 18ms/step - loss: 38.3556 - mean_squared_error: 38.3556 - val_loss: 38.5582 - val_mean_squared_error: 38.5582
Epoch 2/10
11/11 [==============================] - 0s 5ms/step - loss: 38.1666 - mean_squared_error: 38.1666 - val_loss: 38.3390 - val_mean_squared_error: 38.3390
Epoch 3/10
11/11 [==============================] - 0s 4ms/step - loss: 37.9082 - mean_squared_error: 37.9082 - val_loss: 38.0282 - val_mean_squared_error: 38.0282
Epoch 4/10
11/11 [==============================] - 0s 4ms/step - loss: 37.5276 - mean_squared_error: 37.5276 - val_loss: 37.5630 - val_mean_squared_error: 37.5630
Epoch 5/10
11/11 [==============================] - 0s 4ms/step - loss: 36.9500 - mean_squared_error: 36.9500 - val_loss: 36.8718 - val_mean_squared_error: 36.8718
Epoch 6/10
11/11 [==============================] - 0s 4ms/step - loss: 36.1100 - mean_squared_error: 36.1100 - val_loss: 35.8857 - val_mean_squared_error: 35.8857
Epoch 7/10
11/1

GridSearchCV(cv=5,
             estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x7fcf380420d0>,
             param_grid={'batch_size': [50, 100], 'epochs': [10, 25, 50],
                         'optimizador': ['adam', 'rmsprop']},
             scoring='neg_root_mean_squared_error')

In [ ]:
grid_search3.best_params_

{'batch_size': 100, 'epochs': 50, 'optimizador': 'adam'}

In [ ]:
regressor3 = Sequential()
regressor3.add(Dense(input_dim= 8, units=8, kernel_initializer='uniform', activation='softplus'))
regressor3.add(Dense(units=10, kernel_initializer='uniform', activation='softplus'))
regressor3.add(Dense(units=4, kernel_initializer='uniform', activation='softplus'))
regressor3.add(Dense(units=1, kernel_initializer='uniform', activation='relu'))
regressor3.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])
regressor3.fit(X_train, y_train, batch_size=100, validation_split=0.2, verbose=1, epochs=50)

Epoch 1/50
7/7 [==============================] - 1s 26ms/step - loss: 38.2378 - mean_squared_error: 38.2378 - val_loss: 36.8149 - val_mean_squared_error: 36.8149
Epoch 2/50
7/7 [==============================] - 0s 5ms/step - loss: 37.8942 - mean_squared_error: 37.8942 - val_loss: 36.4717 - val_mean_squared_error: 36.4717
Epoch 3/50
7/7 [==============================] - 0s 6ms/step - loss: 37.5322 - mean_squared_error: 37.5322 - val_loss: 36.1119 - val_mean_squared_error: 36.1119
Epoch 4/50
7/7 [==============================] - 0s 7ms/step - loss: 37.1510 - mean_squared_error: 37.1510 - val_loss: 35.7254 - val_mean_squared_error: 35.7254
Epoch 5/50
7/7 [==============================] - 0s 5ms/step - loss: 36.7353 - mean_squared_error: 36.7353 - val_loss: 35.2991 - val_mean_squared_error: 35.2991
Epoch 6/50
7/7 [==============================] - 0s 7ms/step - loss: 36.2754 - mean_squared_error: 36.2754 - val_loss: 34.8187 - val_mean_squared_error: 34.8187
Epoch 7/50
7/7 [===========

In [ ]:
path = '/content/drive/MyDrive/Statistical2/Lab2/' 
regressor3.save(path + '8_10_4_adam.h5')

In [ ]:
model_3 = keras.models.load_model(path + '8_10_4_adam.h5')

In [ ]:
y_preds3 = model_3.predict(X_test)
print(mean_squared_error(y_test, y_preds3))

8.890000981774053


### Estructura 4

In [ ]:
# Estructura 4
def modelo(optimizador):
  regressor4 = Sequential()
  regressor4.add(Dense(input_dim= 8, units=8, kernel_initializer='normal', activation='softmax'))
  regressor4.add(Dense(units=4, kernel_initializer='normal', activation='softmax'))
  regressor4.add(Dense(units=2, kernel_initializer='normal', activation='softmax'))
  regressor4.add(Dense(units=1, kernel_initializer='normal', activation='relu'))
  regressor4.compile(optimizer=optimizador, loss='mean_squared_error', metrics=['mean_squared_error'])
  return regressor4

In [ ]:
resultado4 = KerasRegressor(modelo, validation_split=0.2, verbose=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  """Entry point for launching an IPython kernel.


In [ ]:
grid_search4 = GridSearchCV(estimator= resultado4, 
                         param_grid= parametros, 
                         scoring='neg_root_mean_squared_error', 
                         cv=5)

In [ ]:
grid_search4.fit(X_train, y_train, verbose = 1)

Epoch 1/10
11/11 [==============================] - 1s 17ms/step - loss: 38.4238 - mean_squared_error: 38.4238 - val_loss: 38.7266 - val_mean_squared_error: 38.7266
Epoch 2/10
11/11 [==============================] - 0s 4ms/step - loss: 38.4238 - mean_squared_error: 38.4238 - val_loss: 38.7266 - val_mean_squared_error: 38.7266
Epoch 3/10
11/11 [==============================] - 0s 4ms/step - loss: 38.4238 - mean_squared_error: 38.4238 - val_loss: 38.7266 - val_mean_squared_error: 38.7266
Epoch 4/10
11/11 [==============================] - 0s 4ms/step - loss: 38.4238 - mean_squared_error: 38.4238 - val_loss: 38.7266 - val_mean_squared_error: 38.7266
Epoch 5/10
11/11 [==============================] - 0s 5ms/step - loss: 38.4238 - mean_squared_error: 38.4238 - val_loss: 38.7266 - val_mean_squared_error: 38.7266
Epoch 6/10
11/11 [==============================] - 0s 6ms/step - loss: 38.4238 - mean_squared_error: 38.4238 - val_loss: 38.7266 - val_mean_squared_error: 38.7266
Epoch 7/10
11/1

GridSearchCV(cv=5,
             estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x7fcf3a359090>,
             param_grid={'batch_size': [50, 100], 'epochs': [10, 25, 50],
                         'optimizador': ['adam', 'rmsprop']},
             scoring='neg_root_mean_squared_error')

In [ ]:
grid_search4.best_params_

{'batch_size': 50, 'epochs': 50, 'optimizador': 'rmsprop'}

In [ ]:
regressor4 = Sequential()
regressor4.add(Dense(input_dim= 8, units=8, kernel_initializer='uniform', activation='softmax'))
regressor4.add(Dense(units=4, kernel_initializer='uniform', activation='softmax'))
regressor4.add(Dense(units=2, kernel_initializer='uniform', activation='softmax'))
regressor4.add(Dense(units=1, kernel_initializer='uniform', activation='relu'))
regressor4.compile(optimizer='rmsprop', loss='mean_squared_error', metrics=['mean_squared_error'])
regressor4.fit(X_train, y_train, batch_size=50, validation_split=0.2, verbose=1, epochs=50)

Epoch 1/50
13/13 [==============================] - 1s 16ms/step - loss: 38.7052 - mean_squared_error: 38.7052 - val_loss: 37.2611 - val_mean_squared_error: 37.2611
Epoch 2/50
13/13 [==============================] - 0s 5ms/step - loss: 38.3453 - mean_squared_error: 38.3453 - val_loss: 36.9595 - val_mean_squared_error: 36.9595
Epoch 3/50
13/13 [==============================] - 0s 3ms/step - loss: 38.0456 - mean_squared_error: 38.0456 - val_loss: 36.6778 - val_mean_squared_error: 36.6778
Epoch 4/50
13/13 [==============================] - 0s 5ms/step - loss: 37.7558 - mean_squared_error: 37.7558 - val_loss: 36.3969 - val_mean_squared_error: 36.3969
Epoch 5/50
13/13 [==============================] - 0s 3ms/step - loss: 37.4677 - mean_squared_error: 37.4677 - val_loss: 36.1182 - val_mean_squared_error: 36.1182
Epoch 6/50
13/13 [==============================] - 0s 4ms/step - loss: 37.1807 - mean_squared_error: 37.1807 - val_loss: 35.8416 - val_mean_squared_error: 35.8416
Epoch 7/50
13/1

In [ ]:
path = '/content/drive/MyDrive/Statistical2/Lab2/' 
regressor4.save(path + '8_4_2_rmsprop.h5')

In [ ]:
model_4 = keras.models.load_model(path + '8_4_2_rmsprop.h5')

In [ ]:
y_preds4 = model_4.predict(X_test)
print(mean_squared_error(y_test, y_preds4))

25.87811972037789


## Conclusion Resultados

---
#### Como parámetros se utilizaron batch size de 50 y 100; optimazadores Adam y rmspror; y epoch de 10, 25 y 50. De las 4 estructuras realizadas el mejor resultado es la tercera estructura con un inicializador 'normal', activation 'selu', una capa inicial de 8 neuronas, dos capas ocultas de 10 y 4 neuronas respectivamente y la salida 'relu'. Los mejores parámetros en este modelo según la evaluación son el optimizador Adam, batch size de 100 y 50 epochs.